In [18]:
#!python -m pip install pyserini --user
#!python -m pip install jsonlines --user

In [36]:
!ls -l --block-size=M /mnt/data/factcheck/fever/data-en-latest/

total 26835M
drwxr-xr-x   3 drchajan k13136        1M Sep 10 11:25 emb
-rw-r--r--   1 drchajan factcheck  5972M Aug  3 18:20 enwiki.jsonl
-rw-r--r--   1 drchajan factcheck 14103M Aug  3 17:12 enwiki.xml
drwxr-xr-x 144 drchajan factcheck     1M Jul 29 22:49 extracted
drwxr-xr-x   2 drchajan factcheck     1M Aug 12 17:04 fever
drwxr-xr-x   2 drchajan factcheck     1M Aug  5 12:16 fever-data
drwxr-xr-x   2 drchajan factcheck     1M Aug  5 09:45 index
-rw-r--r--   1 drchajan factcheck     1M Aug  5 20:44 info.txt
drwxr-xr-x   2 drchajan k13136        1M Sep 10 11:42 predictions
drwxr-xr-x   2 drchajan factcheck     0M Aug  6 12:09 wiki-pages


In [37]:
!cat /mnt/data/factcheck/fever/data-en-latest/info.txt

This is EN Wiki dataset based on latest Wiki snapshot 
 - abstracts only (as in original)
 - PTB tokenization using NLTK + fixes (mainly bracket tockenization) using Jupyter notebook (TODO: add to main code-base)
 - note that some pages were renamed since original Fever Wiki snapshot
 
Updates:
05/08/2020 - fixed titles to match original Fever tokenization



# Run on FEVER data

In [1]:
DATAFOLDER='/home/ryparmar/pyserini/fever/'
!ls /home/ryparmar/pyserini/fever/

__MACOSX  license.html	paper_dev.jsonl  train.jsonl  wiki-pages


### Prepare the wiki dump into format suitable for anserini

In [2]:
!python /home/ryparmar/pyserini/src/convert_collection_to_jsonl.py \
    --collection_folder /home/ryparmar/pyserini/fever/wiki-pages \
    --output_folder /home/ryparmar/pyserini/fever/data \
    --max_docs_per_file 10000000 \
    --granularity 'paragraph'

Converting collection...
Converted 100000 docs in 1 files
Converted 200000 docs in 1 files
Converted 300000 docs in 1 files
Converted 400000 docs in 1 files
Converted 500000 docs in 1 files
Converted 600000 docs in 1 files
Converted 700000 docs in 1 files
Converted 800000 docs in 1 files
Converted 900000 docs in 1 files
Converted 1000000 docs in 1 files
Converted 1100000 docs in 1 files
Converted 1200000 docs in 1 files
Converted 1300000 docs in 1 files
Converted 1400000 docs in 1 files
Converted 1500000 docs in 1 files
Converted 1600000 docs in 1 files
Converted 1700000 docs in 1 files
Converted 1800000 docs in 1 files
Converted 1900000 docs in 1 files
Converted 2000000 docs in 1 files
Converted 2100000 docs in 1 files
Converted 2200000 docs in 1 files
Converted 2300000 docs in 1 files
Converted 2400000 docs in 1 files
Converted 2500000 docs in 1 files
Converted 2600000 docs in 1 files
Converted 2700000 docs in 1 files
Converted 2800000 docs in 1 files
Converted 2900000 docs in 1 file

In [22]:
!ls -l --block-size=M /home/ryparmar/pyserini/data

total 8101M
-rw-r--r-- 1 ryparmar k13136 2981M Sep  8 13:27 docs00-id=int.json
-rw-r--r-- 1 ryparmar k13136 3078M Sep  8 19:53 docs00.json


### Try to compute index

In [3]:
!python -m pyserini.index \
    -collection JsonCollection \
    -generator DefaultLuceneDocumentGenerator \
    -threads 1 \
    -input /home/ryparmar/pyserini/fever/data \
    -index /home/ryparmar/pyserini/fever/indexes \
    -storePositions -storeDocvectors -storeRaw

2020-09-10 15:42:44,549 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to INFO
2020-09-10 15:42:44,552 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Starting indexer...
2020-09-10 15:42:44,552 INFO  [main] index.IndexCollection (IndexCollection.java:640) - ============ Loading Parameters ============
2020-09-10 15:42:44,552 INFO  [main] index.IndexCollection (IndexCollection.java:641) - DocumentCollection path: /home/ryparmar/pyserini/fever/data
2020-09-10 15:42:44,553 INFO  [main] index.IndexCollection (IndexCollection.java:642) - CollectionClass: JsonCollection
2020-09-10 15:42:44,553 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Generator: DefaultLuceneDocumentGenerator
2020-09-10 15:42:44,554 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Threads: 1
2020-09-10 15:42:44,554 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Stemmer: porter
2020-09-10 15:42:44,554 INFO  [main] index.I

In [4]:
!ls /home/ryparmar/pyserini/indexes

ls: cannot access /home/ryparmar/pyserini/indexes: No such file or directory


### Perform Retrieval on the Dev

In [5]:
# Generate queries and qrels files for the dev split
!python /home/ryparmar/pyserini/src/generate_queries_and_qrels.py \
--dataset_file /home/ryparmar/pyserini/fever/paper_dev.jsonl \
--output_queries_file /home/ryparmar/pyserini/fever/queries.par.dev.tsv \
--output_qrels_file /home/ryparmar/pyserini/fever/qrels.par.dev.tsv \
--granularity paragraph

Generating qrels...
Generating queries...
Done!


In [6]:
# claim_id, claim
!head -n 3 /home/ryparmar/pyserini/fever/queries.par.dev.tsv

91198	Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.
194462	Tilda Swinton is a vegan.
137334	Fox 2000 Pictures released the film Soul Food.


In [7]:
# claim_id, doc_id (where the evidence is present), evidence?? in the code is 2 (directly number)
!head -n 3 /home/ryparmar/pyserini/fever/qrels.par.dev.tsv

137334	0	Soul_Food_-LRB-film-RRB-	2
111897	0	Telemundo	2
111897	0	Hispanic_and_Latino_Americans	2


In [8]:
!head -n 3 /home/ryparmar/pyserini/fever/paper_dev.jsonl

{"id": 91198, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.", "evidence": [[[108548, null, null, null]]]}
{"id": 194462, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Tilda Swinton is a vegan.", "evidence": [[[227768, null, null, null]]]}
{"id": 137334, "verifiable": "VERIFIABLE", "label": "SUPPORTS", "claim": "Fox 2000 Pictures released the film Soul Food.", "evidence": [[[289914, 283015, "Soul_Food_-LRB-film-RRB-", 0]], [[291259, 284217, "Soul_Food_-LRB-film-RRB-", 0]], [[293412, 285960, "Soul_Food_-LRB-film-RRB-", 0]], [[337212, 322620, "Soul_Food_-LRB-film-RRB-", 0]], [[337214, 322622, "Soul_Food_-LRB-film-RRB-", 0]]]}


In [12]:
# Retrieval run
# k1=0.6 n=0.5 BM25 parameters found by finetuning
!python /home/ryparmar/pyserini/src/retrieve.py \
--hits 1000 --threads 1 \
--index /home/ryparmar/pyserini/fever/indexes \
--queries /home/ryparmar/pyserini/fever/queries.par.dev.tsv \
--output /home/ryparmar/pyserini/fever/run.fever-par-finetuned-paper.dev.tsv \
--k1 0.6 --b 0.5

Initializing BM25, setting k1=0.6 and b=0.5
Retrieving query 0 (0.885 s/query)
Retrieving query 100 (0.112 s/query)
Retrieving query 200 (0.101 s/query)
Retrieving query 300 (0.096 s/query)
Retrieving query 400 (0.094 s/query)
Retrieving query 500 (0.093 s/query)
Retrieving query 600 (0.092 s/query)
Retrieving query 700 (0.091 s/query)
Retrieving query 800 (0.090 s/query)
Retrieving query 900 (0.089 s/query)
Retrieving query 1000 (0.088 s/query)
Retrieving query 1100 (0.088 s/query)
Retrieving query 1200 (0.087 s/query)
Retrieving query 1300 (0.086 s/query)
Retrieving query 1400 (0.086 s/query)
Retrieving query 1500 (0.086 s/query)
Retrieving query 1600 (0.086 s/query)
Retrieving query 1700 (0.086 s/query)
Retrieving query 1800 (0.086 s/query)
Retrieving query 1900 (0.086 s/query)
Retrieving query 2000 (0.086 s/query)
Retrieving query 2100 (0.086 s/query)
Retrieving query 2200 (0.086 s/query)
Retrieving query 2300 (0.086 s/query)
Retrieving query 2400 (0.085 s/query)
Retrieving query 2

In [15]:
# claim_id, predicted_document_id, rank  -- see that there up to 1000 hits (hits arg)
!head -n 1005 /home/ryparmar/pyserini/fever/run.fever-par-finetuned-paper.dev.tsv | tail -n 10

91198	Herm_Edwards	996
91198	Max_Choboian	997
91198	1970_Cincinnati_Bengals_season	998
91198	Scott_Hilton	999
91198	1970_San_Francisco_49ers_season	1000
194462	Tilda	1
194462	Swinton_-LRB-surname-RRB-	2
194462	List_of_awards_and_nominations_received_by_Tilda_Swinton	3
194462	Julia_-LRB-2008_film-RRB-	4
194462	Edinburgh_Filmhouse	5


### Evaluation

In [21]:
import jsonlines
# Convert .tsv (anserini) prediction into .jsonl (drqa) predictions
def convert_to_drqa_format(inputfile, outputfile, truthfile):
    out = {}
    with jsonlines.open(truthfile) as ft:
        for line in ft.iter():
            out[str(line['id'])] = {'id': line['id'],
                               'label': line['label'],
                               'evidence': line['evidence']}
    with open(inputfile) as fr:
        for line in fr.readlines():
            claim_id, pred_docid, _ = line.split('\t')
            if claim_id in out and 'predicted_pages' in out[claim_id]:
                out[claim_id]['predicted_pages'].append(pred_docid)
            else:
                out[claim_id]['predicted_pages'] = [pred_docid]

        with jsonlines.open(outputfile, 'w') as fw:
            for key in out:
                fw.write(out[key])
                
# Convert .jsonl (drqa) predictions into .tsv (anserini) prediction 
def convert_to_anserini_format(inputfile, outputfile):
    out = []
    with jsonlines.open(inputfile) as ft:
        for line in ft.iter():
            for rank, docid in enumerate(line['predicted_pages']):
                out.append("{}\t{}\t{}\n".format(line['id'], docid, rank+1))

        with open(outputfile, 'w') as fw:
            for line in out:
                fw.write(line)

In [14]:
# Evaluate anserini finetuned on FEVER dev data
!python /home/ryparmar/pyserini/src/evaluate_doc_retrieval.py \
--truth_file /home/ryparmar/pyserini/fever/paper_dev.jsonl \
--run_file /home/ryparmar/pyserini/fever/run.fever-par-finetuned-paper.dev.tsv

k	Fully Supported	Oracle Accuracy
1	0.3857	0.5905
5	0.6367	0.7578
10	0.7193	0.8129
25	0.8003	0.8669
50	0.8473	0.8982
100	0.8804	0.9203
500	0.9320	0.9547


In [167]:
# Evaluate with trec_eval (TREC)
# Convert the runs and qrels to trec files
!python /home/ryparmar/pyserini/src/convert_msmarco_to_trec_run.py \
--input /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.tsv \
--output /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.trec

!python /home/ryparmar/pyserini/src/convert_msmarco_to_trec_qrels.py \
--input /home/ryparmar/pyserini/qrels.par.dev.tsv \
--output /home/ryparmar/pyserini/qrels.par.dev.trec

Done!
Done!


In [169]:
# Run evaluation itself
!./home/ryparmar/pyserini/src/trec_eval.9.0.4/trec_eval -c -m all_trec \
/home/ryparmar/pyserini/qrels.par.dev.trec /home/ryparmar/pyserini/run.fever-par-0.6-0.5.dev.trec

/usr/bin/sh: ./home/ryparmar/pyserini/src/trec_eval.9.0.4/trec_eval: No such file or directory


# Run the DrQA baseline

In [42]:
# !python -m pip install allennlp==0.9.0 --user
# !python -m pip install fever-api --user
# !python -m pip install fever-allennlp --user
# !python -m pip install fever-drqa --user
# !python -m pip install pexpect --user

In [43]:
!ls /mnt/data/factcheck/fever/data-en-latest/index

fever-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz


In [8]:
!python /home/ryparmar/drchajan/src/predict_pages_drqa.py \
--model /mnt/data/factcheck/fever/data-en-latest/index/fever-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz \
--database /mnt/data/factcheck/fever/data-en-latest/fever/fever.db \
--in-file /mnt/data/factcheck/fever/data-en-latest/fever-data/dev.jsonl \
--out-file /home/ryparmar/pyserini/dev-predict-drqa \
--max-page 5

Traceback (most recent call last):
  File "/home/ryparmar/drchajan/src/predict_pages_drqa.py", line 11, in <module>
    from drqa.retriever.top_n import TopNDocsTopNSents
ModuleNotFoundError: No module named 'drqa.retriever.top_n'


In [204]:
!cat /home/ryparmar/pyserini/dev-predict-drqa

In [11]:
!python -m predict_drqa ./venv

/mnt/appl/software/Python/3.7.4-GCCcore-8.3.0/bin/python: No module named predict_drqa
